In [115]:
 !pip install wget
 !pip install opencv-python

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [1]:
seed=42

In [2]:
import os
from os.path import exists
import sys
import requests
import zipfile
import wget
cwd = os.getcwd()
train_dir='inaturalist_12K/train/' 
test_dir='inaturalist_12K/val/'
savePath=cwd+'/DataSet/'
classes=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [3]:
file_exists = exists(cwd+'/DataSet/nature_12K.zip')
if(file_exists==False):
    url='https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
    print('Downloading..')
    wget.download(url,savePath)
    
extract_exists = exists(cwd+'/DataSet/inaturalist_12K')   
if(extract_exists==False):   
    savePath=cwd+'\\DataSet'
    savefile=cwd+'\\DataSet\\nature_12K.zip'
    print('Extracting..')
    with zipfile.ZipFile(savefile, 'r') as zip_ref:
        zip_ref.extractall(savePath)
    print('Finished..')
   
    

### Download and Extract 

In [4]:
train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'
classes=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import cv2
imagePath=os.getcwd()+'\\DataSet\\inaturalist_12K\\train'
listImages=[]
columns = 5
rows = 5
imcount=1

for i in classes:
    p=imagePath+'\\'+i
    files=os.listdir( p)
    img=random.choice(files)
    imgplt=mpimg.imread( p+'\\'+str(img))
    imgplt=cv2.resize(imgplt,(128,128)) 
    listImages.append(imgplt)
    #plt.imshow(imgplt)
    #fig.add_subplot(rows, columns, imcount)
    #imcount+=1
    #plt.axis('off')
    #plt.title(i) 

_, axs = plt.subplots(1, 10, figsize=(15, 15))
axs = axs.flatten()
for img, ax,cls in zip(listImages, axs,classes):
  
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(str(cls))
plt.show()

<Figure size 1500x1500 with 10 Axes>

In [6]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from keras.layers import Conv2D , MaxPool2D,MaxPooling3D , Flatten , Dropout, Dense, Activation, BatchNormalization

In [7]:

def dataset(seed):
    cwd=os.getcwd()
    imagesize=(64,64)
    batchsize=32
    train_dir = cwd+'\\DataSet\\inaturalist_12K\\train\\'
    test_dir = cwd+'\\DataSet\\inaturalist_12K\\val\\'
    print('Training Dataset')
    def scale(image, label):
        return tf.image.convert_image_dtype(image*(1.0/255), tf.float32), label

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
         label_mode ='categorical',
        validation_split=0.1,
        subset="training",
        color_mode='rgb',
        seed=seed,
        image_size=imagesize,
        batch_size=batchsize
    )
    #train_ds=train_ds.map(scale)
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
        validation_split=0.1,
        subset="validation",
        seed=seed,
        color_mode="rgb",
        label_mode ='categorical',
        image_size=imagesize,
        batch_size=batchsize,
    )
    #val_ds=val_ds.map(scale)
    print('')
    print('Test Dataset')
    test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,batch_size=batchsize, shuffle=True,image_size=imagesize,label_mode='categorical',
        color_mode='rgb')
    #test_ds=test_ds.map(scale)
    return train_ds,val_ds, test_ds

In [8]:
 
def Import_dataset(seed,augment_data=False):
    cwd=os.getcwd()
    imagesize=(128,128)
    batchsize=32
    image_size=128
    train_dir = cwd+'\\DataSet\\inaturalist_12K\\train\\'
    test_dir = cwd+'\\DataSet\\inaturalist_12K\\val\\'
    print('Training Dataset')
    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=90,
                                      zoom_range=0.2,
                                      shear_range=0.2,
                                      validation_split=0.1,
                                      horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)
    else:
        train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_ds = train_datagen.flow_from_directory(train_dir,subset="training", color_mode='rgb',target_size=(image_size, image_size), batch_size=batchsize)
    val_ds = train_datagen.flow_from_directory(train_dir,subset="validation", color_mode='rgb',target_size=(image_size, image_size), batch_size=batchsize)
    print('')
    print('Test Dataset')
    test_ds = test_datagen.flow_from_directory(test_dir, target_size=(128, 128),color_mode='rgb',batch_size=30)
   
    
    
    
  
    return train_ds,val_ds, test_ds

### Model consisting of 5 convolution layers

In [9]:
def CNN(filters, size_of_filters,\
        activation_function,\
        number_of_neurons_in_the_dense_layer,\
        number_of_classes,\
        dropout,\
        BatchNormalization=True):
    model = Sequential()
    num_of_filters=len(filters)
    model.add(Conv2D(no_of_filters[0], size_of_filters[0],input_shape=(128,128,3),kernel_initializer=tf.keras.initializers.GlorotNormal(seed=42),activation=activation_function[0]))     
    for i in range(num_of_filters-1):
        model.add(Conv2D(no_of_filters[i+1], size_of_filters[i+1],kernel_initializer=tf.keras.initializers.GlorotNormal(seed=42)))
        if BatchNormalization:
            model.add(tf.keras.layers.BatchNormalization(momentum= 0.99))
        model.add(Activation(activation_function[i]))
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(dropout))
       
    model.add(Flatten())

    model.add(Dense(number_of_neurons_in_the_dense_layer,activation=activation_function[-2])) 
    model.add(Dropout(dropout))
    model.add(Dense(number_of_classes, activation=activation_function[-1]))
    return model



In [10]:
train_ds,val_ds,test_ds=Import_dataset(seed=42)

Training Dataset
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.

Test Dataset
Found 2000 images belonging to 10 classes.


In [11]:
no_of_filters = [64,64,64,64,64]
size_of_filters = [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
activation_function = ['relu','relu','relu','relu','relu','relu','softmax']
number_of_neurons_in_the_dense_layer = 256
dropout=0.3
number_of_classes=10
model=CNN(no_of_filters,size_of_filters,activation_function,number_of_neurons_in_the_dense_layer,number_of_classes,dropout) 

In [98]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=[tf.keras.losses.CategoricalCrossentropy()], metrics=['accuracy'])
hist=model.fit(train_ds, epochs=4,validation_data=val_ds)

Epoch 1/4
282/282 [==============================] - 731s 3s/step - loss: 2.3628 - accuracy: 0.1600 - val_loss: 2.4389 - val_accuracy: 0.1051
Epoch 2/4
282/282 [==============================] - 705s 2s/step - loss: 2.1448 - accuracy: 0.2119 - val_loss: 2.1709 - val_accuracy: 0.2112
Epoch 3/4
282/282 [==============================] - 679s 2s/step - loss: 2.1109 - accuracy: 0.2378 - val_loss: 2.1896 - val_accuracy: 0.2142
Epoch 4/4
282/282 [==============================] - 646s 2s/step - loss: 2.0489 - accuracy: 0.2630 - val_loss: 2.1961 - val_accuracy: 0.2262


In [ ]:
model.save(cwd+'\\models\\')

In [ ]:
model=keras.models.load_model(cwd+'\\models\\')

In [ ]:
from sklearn.metrics import accuracy_score
prediction=model.predict(test_ds)
predictionz=np.argmax(prediction,axis=1)
accuracy_score(test_ds.labels,predictionz)